# Clustering

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [2]:
import sys

if ('..' not in sys.path):
    print('append .. to sys.path')
    sys.path.append('..')
print(sys.path)

import numpy as np
import scipy.sparse
# import sparsechem as sc
import argparse
import leader_follower as lf

append .. to sys.path
['/home/kbardool/WSL-projs/Chembl-pipeline', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python39.zip', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/lib-dynload', '', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/IPython/extensions', '/home/kbardool/.ipython', '..']


In [3]:
for i in sys.path:
    print(i)

/home/kbardool/WSL-projs/Chembl-pipeline
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python39.zip
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/lib-dynload

/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/IPython/extensions
/home/kbardool/.ipython
..


In [4]:
def print_dbg(text, verbose = False):
    if verbose:
        print(text)

In [5]:
def load_sparse(filename):
    """Loads sparse from Matrix market or Numpy .npy file."""
    if filename is None:
        return None
    if filename.endswith('.mtx'):
        return scipy.io.mmread(filename).tocsr()
    elif filename.endswith('.npy'):
        return np.load(filename, allow_pickle=True).item().tocsr()
    elif filename.endswith('.npz'):
        return scipy.sparse.load_npz(filename).tocsr()
    raise ValueError(f"Loading '{filename}' failed. It must have a suffix '.mtx', '.npy', '.npz'.")


In [6]:
def hierarchical_clustering(X, dists):
    """
    Args:
        X       compound matrix in CSR
        dists   list of (increasing) distances
    Sequentially clusters with each dists, returns final cluster ids
    """
    assert type(X) == scipy.sparse.csr.csr_matrix, "X should be csr_matrix (scipy.sparse)"
    
    print(f"X shape: {X.shape}")
    cl0 = np.arange(X.shape[0])
    Xc  = X

    for dist in dists:
        print(f"Running clustering for distance: {dist}")
        cl, cent = cluster(Xc, dist)
        Xc       = Xc[cent]
        cl0      = cl[cl0]

    return cl0


In [7]:
%load_ext Cython
# del cluster

In [8]:
%%cython --cplus --force

from libcpp.vector cimport vector
import numpy as np

def cluster(X, radius, verbose = False):
    
    cdef int i, j, k, row, col, closest
    cdef double min_dist, tmp_dist
    cdef int num_changed

    cdef int [:] x_indptr  = X.indptr
    cdef int [:] x_indices = X.indices
#     cdef double [:] x_data = X.data

    cdef int X_shape0 = X.shape[0]
    cdef int X_shape1 = X.shape[1]


    cdef vector[double]      dist
    cdef vector[vector[int]] centers
    cdef vector[double]      Cnorm
    cdef vector[int]         center_ids

    ## Xnorm[]: no of non-zero columns in each row
    cdef long   [:] Xnorm = np.array((X != 0).sum(axis = 1)).flatten()
    
    ## dists[]:
    cdef double [:] dists = np.zeros(X_shape0, dtype=np.float64)

    ## Clusters[] : cluster assignment of each X row
    cdef long [:] clusters = np.zeros(X.shape[0], dtype = np.int64) - 1
    
    ## perm_ids[] : Random permutation of row numbers to provide randomize selection of rows 
    cdef long [:] perm_ids = np.random.permutation(X.shape[0])       

    
    print( f"X shape        :  {X.shape}")
    print( f"X.indptr shape :  {X.indptr.shape}")
    print( f"X.indices shape:  {X.indices.shape}")    
#     print(f" ==== perm ids:       {np.asarray(perm_ids)[0:100]}")

    print(f" ==== Xnorm shape:     {Xnorm.shape}")
    print(f" ==== Xnorm [:50]:     {np.asarray(Xnorm)[:50]}" )
    print()

    print(f" ==== dists size:      {dists.shape}")
    print(f" ==== dists[:50]:      {np.asarray(dists)[:50]}")
    print()

    print(f" ==== dist size:       {dist.size()}")
    print(f" ==== dist[:50]:       {np.asarray(dist)[:50]}")    
    print()
    
    print(f" ==== centers size     {centers.size()}")
    print(f" ==== centers[:50]:    {np.asarray(centers)[:50]}")     
    centers.resize(X_shape1)
    print(f" ==== centers size     {centers.size()}")
    print(f" ==== centers[:50]:    {np.asarray(centers)[:50]}") 
    
    print()
    print(f" ==== Cnorm size       {Cnorm.size()}")  
    print(f" ==== Cnorm :          {np.asarray(Cnorm)[:50]}") 
    print()
    
    print(f" ==== center_ids size: {center_ids.size()}")    
    print(f" ==== center_ids[:50]: {np.asarray(center_ids)[:50]}")    
    print()    
    
    print(f" ==== clusters size:   {clusters.shape}")
    print(f" ==== clusters[:50]:   {np.asarray(clusters)[:50]}")
    print()    
    
    print(f" ==== perm_ids size:   {perm_ids.shape}")
    print(f" ==== perm_ids :       {np.asarray(perm_ids)[:10]}. . . . .")

    ## algorithm part

    print("=====================================")
    print(" Creating initial Cluster Centroids ")
    print("=====================================")  
    
    for i in range(X_shape0):
#         if  i > 10:
#             return
        if i % 10000 == 0:
            print(f"Row {i}.")
                
        row = perm_ids[i]
#         if verbose:
#             print(f">> i: {i:2d}/{X_shape0} - row = perm_ids[i]: {perm_ids[i]:6d}  - x_indptr[{row:6d}]: "
#                   f"{x_indptr[row]:9d}  x_indptr[{row+1:6d}]: {x_indptr[row+1]:9d}  size(Cnorm): {Cnorm.size()}")
        
        ## computing distances to all centers
        dists[0 : Cnorm.size()] = 0.0
        
        ## for current row, get column indices and populated columns data for current row 
        
        for j in range(x_indptr[row], x_indptr[row+1]):
            col = x_indices[j]
#             if verbose:
#                 print(f"    2.1  row: {row} col= x_indices[{j}] = {col}   k= size(centers[{col}]): {centers[col].size()}")
            for k in range(centers[col].size()):
                dists[ centers[col][k] ] += 1.0
#                 if verbose:
#                     print(f"    2.2  set dists[centers[{col}][{k}]] = dists[{centers[col][k]}] += 1 -> dists[{centers[col][k]}] = {dists[ centers[col][k] ]}")
                
                
        closest = -1
        min_dist = radius
        
        ## Calculate Distance from Centroids 
        for j in range(Cnorm.size()):
#             if verbose:
#                 print(f"   3.1 - j: {j} / {Cnorm.size()} : Xnorm[{row}]: {Xnorm[row]} -  dists[{j}] = 1.0 - {dists[j]} / ({Xnorm[row]} + {Cnorm[j]} - {dists[j]})"\
#                       f" = { 1.0 - dists[j] / (Xnorm[row] + Cnorm[j] - dists[j]):.4f}")            
            dists[j] = 1.0 - dists[j] / (Xnorm[row] + Cnorm[j] - dists[j])
            if dists[j] < min_dist:
#                 if verbose:
#                     print(f"   3.2 - {dists[j]} < {min_dist} - closest set to {j} ")                         
                min_dist = dists[j]
                closest  = j
        
        # If a centroid was found within min_distance, assign row to that centroid
        # and move on to next row
        if closest >= 0:
            clusters[row] = closest
            continue

        ## Otherwise, create a new cluster
#         if verbose:
#             print(f"   4 - Create a new cluster k = size(Cnorm): {Cnorm.size()}     Xnorm[row] : {Xnorm[row]}")
        k = Cnorm.size()
        
        for j in range(x_indptr[row], x_indptr[row+1]):
            # Add new element at end of centers
            centers[ x_indices[j] ].push_back(k)
#             if verbose:
#                 print(f"      4.2 - j: {j} - :  Centers[ {x_indices[j]} ].push_back({k})    add k = Cnorm.size() = {k} "\
#                   f"to centers[{x_indices[j]}]  length(centers[ {x_indices[j] }]): {centers[ x_indices[j] ].size()}")
                
        clusters[row] = k
        Cnorm.push_back(Xnorm[row])
        center_ids.push_back(row)
#         if verbose:
#             print(f"       clusters[{row}] = {k}")
#             print(f"       Cnorm.add(Xnorm[{row}] = {Xnorm[row]}):  new size(Cnorm):{Cnorm.size()}")
#             print("      ", Cnorm)
#             print(f"       center_ids.add(row: {row})    new size(center_ids):{center_ids.size()}")
#             print("      ", center_ids)
        
    print("==============================================")
    print("Reassigning compounds to the closest clusters.")
    print("==============================================") 
    num_changed = 0
    
    for row in range(X_shape0):

        if row % 50000 == 0:
            print(f"Row {row}.")
            
        ## compute distances to all clusters, assign to the closest
        for j in range(x_indptr[row], x_indptr[row+1]):
            col = x_indices[j]
            for k in range(centers[col].size()):
                dists[ centers[col][k] ] += 1.0

        closest = -1
        min_dist = radius + 1e-5
        
        # iterate over centroids, calculate distance to Clusters
        for j in range(Cnorm.size()):
            try:
                tmp_dist = 1.0 - dists[j] / (Xnorm[row] + Cnorm[j] - dists[j])
            except ZeroDivisionError:
                print(f" Zero Division Error Encountered  - row:{row}   j: {j}"\
                      f" Xnorm[{row}]:   {Xnorm[row]}    Cnorm[{j}]:   {Cnorm[j]}      dists[{j}]:   {dists[j]}")
            else:
                if tmp_dist < min_dist:
                    min_dist = tmp_dist
                    closest  = j
                    if min_dist == 0:
                        ## best possible found, distance is zero
                        break
                        
        if (closest >= 0) and (clusters[row] != closest):
            clusters[row]  = closest
            num_changed   += 1

    print(f"Reassignement changed {num_changed} assignments.")
    print(f"Total {len(center_ids)} clusters.")

    return np.asarray(clusters), np.asarray(center_ids)

/home/kbardool/.cache/ipython/cython/_cython_magic_db5b38ef51f684777488661511c5b844.cpp: In function ‘PyObject* __pyx_pf_46_cython_magic_db5b38ef51f684777488661511c5b844_cluster(PyObject*, PyObject*, PyObject*, PyObject*)’:
/home/kbardool/.cache/ipython/cython/_cython_magic_db5b38ef51f684777488661511c5b844.cpp:3826:39: warning: comparison of integer expressions of different signedness: ‘int’ and ‘std::vector<int>::size_type’ {aka ‘long unsigned int’} [-Wsign-compare]
 3826 |       for (__pyx_t_21 = 0; __pyx_t_21 < __pyx_t_23; __pyx_t_21+=1) {
      |                            ~~~~~~~~~~~^~~~~~~~~~~~
/home/kbardool/.cache/ipython/cython/_cython_magic_db5b38ef51f684777488661511c5b844.cpp:3878:37: warning: comparison of integer expressions of different signedness: ‘int’ and ‘std::vector<double>::size_type’ {aka ‘long unsigned int’} [-Wsign-compare]
 3878 |     for (__pyx_t_17 = 0; __pyx_t_17 < __pyx_t_27; __pyx_t_17+=1) {
      |                          ~~~~~~~~~~~^~~~~~~~~~~~
/home/kba

## main

In [9]:
parser = argparse.ArgumentParser(description="Training a multi-task model.")
parser.add_argument("--x"  , help="Descriptor file (matrix market or numpy)", type=str, required=True)
parser.add_argument("--out", help="Output file for the clusters (.npy)", type=str, required=True)
parser.add_argument("--dists", nargs="+", help="Distances", default=[], type=float, required=True)


input_args =  "python ../leader_follower/cluster.py"\
              " --x      output/chembl_29/chembl_29_X.npy " \
              " --out    output/chembl_29/chembl_29_clustering.npy "\
              " --dist   0.5 0.7".split()
print(input_args)

args = parser.parse_args(input_args[2:])
print(vars(args))
dists = args.dists

['python', '../leader_follower/cluster.py', '--x', 'output/chembl_29/chembl_29_X.npy', '--out', 'output/chembl_29/chembl_29_clustering.npy', '--dist', '0.5', '0.7']
{'x': 'output/chembl_29/chembl_29_X.npy', 'out': 'output/chembl_29/chembl_29_clustering.npy', 'dists': [0.5, 0.7]}


In [10]:
print(f"Loading '{args.x}'.")
# X = sc.load_sparse(args.x).tocsr()
X = load_sparse(args.x).tocsr()


X.shape

# Xnorm = np.array((X != 0).sum(axis = 1)).flatten()

# Xnorm.shape


Loading 'output/chembl_29/chembl_29_X.npy'.


(423737, 32000)

### Hierarchical Clustering

In [11]:
# tmp =  np.array((X != 0).sum(axis=1))
# print(tmp.shape)
# tmp = tmp.flatten()
# print(tmp.shape)
# print(type(tmp))
# print(tmp[1000:1200])
# del tmp

In [11]:
print("Clustering.")
# two step clustering, first at 0.5, then at 0.6
cl_hier = hierarchical_clustering(X, dists=args.dists)

Clustering.
X shape: (423737, 32000)
Running clustering for distance: 0.5
X shape        :  (423737, 32000)
X.indptr shape :  (423738,)
X.indices shape:  (32838752,)
 ==== Xnorm shape:     [423737, 0, 0, 0, 0, 0, 0, 0]
 ==== Xnorm [:50]:     [ 79  83  87 185  49 167 127  78  81  97 258  56  44  46  85  54  47 168
  80 197 163 191 113  57  31  86  78 135 108 187  54  85 177  65  62  99
  70 102 257  91 101 189 130  98  51 110  76 112 203  68]

 ==== dists size:      [423737, 0, 0, 0, 0, 0, 0, 0]
 ==== dists[:50]:      [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]

 ==== dist size:       0
 ==== dist[:50]:       []

 ==== centers size     0
 ==== centers[:50]:    []
 ==== centers size     32000
 ==== centers[:50]:    []

 ==== Cnorm size       0
 ==== Cnorm :          []

 ==== center_ids size: 0
 ==== center_ids[:50]: []

 ==== clusters size:   [423737, 0, 0, 0, 0, 0, 0, 0]
 ==== 

 Zero Division Error Encountered  - row:11178   j: 35423 Xnorm[11178]:   63    Cnorm[35423]:   2.0      dists[35423]:   65.0
 Zero Division Error Encountered  - row:11183   j: 35423 Xnorm[11183]:   63    Cnorm[35423]:   2.0      dists[35423]:   65.0
 Zero Division Error Encountered  - row:11208   j: 35423 Xnorm[11208]:   64    Cnorm[35423]:   2.0      dists[35423]:   66.0
 Zero Division Error Encountered  - row:11237   j: 35423 Xnorm[11237]:   64    Cnorm[35423]:   2.0      dists[35423]:   66.0
 Zero Division Error Encountered  - row:11343   j: 35423 Xnorm[11343]:   64    Cnorm[35423]:   2.0      dists[35423]:   66.0
 Zero Division Error Encountered  - row:11381   j: 7675 Xnorm[11381]:   39    Cnorm[7675]:   2.0      dists[7675]:   41.0
 Zero Division Error Encountered  - row:11382   j: 7675 Xnorm[11382]:   39    Cnorm[7675]:   2.0      dists[7675]:   41.0
 Zero Division Error Encountered  - row:11419   j: 35423 Xnorm[11419]:   64    Cnorm[35423]:   2.0      dists[35423]:   66.0
 Zero 

 Zero Division Error Encountered  - row:13295   j: 35423 Xnorm[13295]:   66    Cnorm[35423]:   2.0      dists[35423]:   68.0
 Zero Division Error Encountered  - row:13335   j: 35423 Xnorm[13335]:   66    Cnorm[35423]:   2.0      dists[35423]:   68.0
 Zero Division Error Encountered  - row:13353   j: 35423 Xnorm[13353]:   66    Cnorm[35423]:   2.0      dists[35423]:   68.0
 Zero Division Error Encountered  - row:13367   j: 35423 Xnorm[13367]:   66    Cnorm[35423]:   2.0      dists[35423]:   68.0
 Zero Division Error Encountered  - row:13383   j: 35423 Xnorm[13383]:   66    Cnorm[35423]:   2.0      dists[35423]:   68.0
 Zero Division Error Encountered  - row:13393   j: 35423 Xnorm[13393]:   66    Cnorm[35423]:   2.0      dists[35423]:   68.0
 Zero Division Error Encountered  - row:13402   j: 35423 Xnorm[13402]:   66    Cnorm[35423]:   2.0      dists[35423]:   68.0
 Zero Division Error Encountered  - row:13475   j: 35423 Xnorm[13475]:   66    Cnorm[35423]:   2.0      dists[35423]:   68.0


 Zero Division Error Encountered  - row:15740   j: 35423 Xnorm[15740]:   69    Cnorm[35423]:   2.0      dists[35423]:   71.0
 Zero Division Error Encountered  - row:15761   j: 35423 Xnorm[15761]:   69    Cnorm[35423]:   2.0      dists[35423]:   71.0
 Zero Division Error Encountered  - row:15787   j: 35423 Xnorm[15787]:   69    Cnorm[35423]:   2.0      dists[35423]:   71.0
 Zero Division Error Encountered  - row:15809   j: 35423 Xnorm[15809]:   69    Cnorm[35423]:   2.0      dists[35423]:   71.0
 Zero Division Error Encountered  - row:15824   j: 35423 Xnorm[15824]:   69    Cnorm[35423]:   2.0      dists[35423]:   71.0
 Zero Division Error Encountered  - row:15858   j: 35423 Xnorm[15858]:   70    Cnorm[35423]:   2.0      dists[35423]:   72.0
 Zero Division Error Encountered  - row:15884   j: 7675 Xnorm[15884]:   45    Cnorm[7675]:   2.0      dists[7675]:   47.0
 Zero Division Error Encountered  - row:15889   j: 35423 Xnorm[15889]:   70    Cnorm[35423]:   2.0      dists[35423]:   72.0
 Ze

 Zero Division Error Encountered  - row:17465   j: 7675 Xnorm[17465]:   56    Cnorm[7675]:   2.0      dists[7675]:   58.0
 Zero Division Error Encountered  - row:17478   j: 35423 Xnorm[17478]:   82    Cnorm[35423]:   2.0      dists[35423]:   84.0
 Zero Division Error Encountered  - row:17495   j: 7675 Xnorm[17495]:   56    Cnorm[7675]:   2.0      dists[7675]:   58.0
 Zero Division Error Encountered  - row:17506   j: 35423 Xnorm[17506]:   82    Cnorm[35423]:   2.0      dists[35423]:   84.0
 Zero Division Error Encountered  - row:17518   j: 7675 Xnorm[17518]:   56    Cnorm[7675]:   2.0      dists[7675]:   58.0
 Zero Division Error Encountered  - row:17523   j: 7675 Xnorm[17523]:   56    Cnorm[7675]:   2.0      dists[7675]:   58.0
 Zero Division Error Encountered  - row:17555   j: 7675 Xnorm[17555]:   57    Cnorm[7675]:   2.0      dists[7675]:   59.0
 Zero Division Error Encountered  - row:17591   j: 7675 Xnorm[17591]:   57    Cnorm[7675]:   2.0      dists[7675]:   59.0
 Zero Division Err

...

 Zero Division Error Encountered  - row:70038   j: 24670 Xnorm[70038]:   81    Cnorm[24670]:   1.0      dists[24670]:   82.0
 Zero Division Error Encountered  - row:70041   j: 18967 Xnorm[70041]:   82    Cnorm[18967]:   1.0      dists[18967]:   83.0
 Zero Division Error Encountered  - row:70055   j: 24670 Xnorm[70055]:   81    Cnorm[24670]:   1.0      dists[24670]:   82.0
 Zero Division Error Encountered  - row:70163   j: 18967 Xnorm[70163]:   82    Cnorm[18967]:   1.0      dists[18967]:   83.0
 Zero Division Error Encountered  - row:70163   j: 24670 Xnorm[70163]:   82    Cnorm[24670]:   1.0      dists[24670]:   83.0
 Zero Division Error Encountered  - row:70226   j: 18967 Xnorm[70226]:   82    Cnorm[18967]:   1.0      dists[18967]:   83.0
 Zero Division Error Encountered  - row:70226   j: 24670 Xnorm[70226]:   82    Cnorm[24670]:   1.0      dists[24670]:   83.0
 Zero Division Error Encountered  - row:70289   j: 18967 Xnorm[70289]:   82    Cnorm[18967]:   1.0      dists[18967]:   83.0


 Zero Division Error Encountered  - row:73208   j: 18967 Xnorm[73208]:   83    Cnorm[18967]:   1.0      dists[18967]:   84.0
 Zero Division Error Encountered  - row:73242   j: 24670 Xnorm[73242]:   82    Cnorm[24670]:   1.0      dists[24670]:   83.0
 Zero Division Error Encountered  - row:73264   j: 24670 Xnorm[73264]:   82    Cnorm[24670]:   1.0      dists[24670]:   83.0
 Zero Division Error Encountered  - row:73305   j: 18967 Xnorm[73305]:   83    Cnorm[18967]:   1.0      dists[18967]:   84.0
 Zero Division Error Encountered  - row:73306   j: 18967 Xnorm[73306]:   83    Cnorm[18967]:   1.0      dists[18967]:   84.0
 Zero Division Error Encountered  - row:73379   j: 24670 Xnorm[73379]:   82    Cnorm[24670]:   1.0      dists[24670]:   83.0
 Zero Division Error Encountered  - row:73386   j: 18967 Xnorm[73386]:   83    Cnorm[18967]:   1.0      dists[18967]:   84.0
 Zero Division Error Encountered  - row:73407   j: 18967 Xnorm[73407]:   83    Cnorm[18967]:   1.0      dists[18967]:   84.0


 Zero Division Error Encountered  - row:75902   j: 24670 Xnorm[75902]:   86    Cnorm[24670]:   1.0      dists[24670]:   87.0
 Zero Division Error Encountered  - row:75908   j: 24670 Xnorm[75908]:   86    Cnorm[24670]:   1.0      dists[24670]:   87.0
 Zero Division Error Encountered  - row:75913   j: 24670 Xnorm[75913]:   86    Cnorm[24670]:   1.0      dists[24670]:   87.0
 Zero Division Error Encountered  - row:76040   j: 18967 Xnorm[76040]:   84    Cnorm[18967]:   1.0      dists[18967]:   85.0
 Zero Division Error Encountered  - row:76105   j: 24670 Xnorm[76105]:   86    Cnorm[24670]:   1.0      dists[24670]:   87.0
 Zero Division Error Encountered  - row:76194   j: 18967 Xnorm[76194]:   85    Cnorm[18967]:   1.0      dists[18967]:   86.0
 Zero Division Error Encountered  - row:76345   j: 18967 Xnorm[76345]:   85    Cnorm[18967]:   1.0      dists[18967]:   86.0
 Zero Division Error Encountered  - row:76359   j: 18967 Xnorm[76359]:   85    Cnorm[18967]:   1.0      dists[18967]:   86.0


 Zero Division Error Encountered  - row:79546   j: 18967 Xnorm[79546]:   90    Cnorm[18967]:   1.0      dists[18967]:   91.0
 Zero Division Error Encountered  - row:79581   j: 24670 Xnorm[79581]:   87    Cnorm[24670]:   1.0      dists[24670]:   88.0
 Zero Division Error Encountered  - row:79582   j: 24670 Xnorm[79582]:   87    Cnorm[24670]:   1.0      dists[24670]:   88.0
 Zero Division Error Encountered  - row:79604   j: 18967 Xnorm[79604]:   90    Cnorm[18967]:   1.0      dists[18967]:   91.0
 Zero Division Error Encountered  - row:79631   j: 24670 Xnorm[79631]:   87    Cnorm[24670]:   1.0      dists[24670]:   88.0
 Zero Division Error Encountered  - row:79656   j: 24670 Xnorm[79656]:   87    Cnorm[24670]:   1.0      dists[24670]:   88.0
 Zero Division Error Encountered  - row:79719   j: 24670 Xnorm[79719]:   87    Cnorm[24670]:   1.0      dists[24670]:   88.0
 Zero Division Error Encountered  - row:79740   j: 18967 Xnorm[79740]:   90    Cnorm[18967]:   1.0      dists[18967]:   91.0


 Zero Division Error Encountered  - row:83479   j: 18967 Xnorm[83479]:   92    Cnorm[18967]:   1.0      dists[18967]:   93.0
 Zero Division Error Encountered  - row:83524   j: 24670 Xnorm[83524]:   91    Cnorm[24670]:   1.0      dists[24670]:   92.0
 Zero Division Error Encountered  - row:83623   j: 18967 Xnorm[83623]:   92    Cnorm[18967]:   1.0      dists[18967]:   93.0
 Zero Division Error Encountered  - row:83632   j: 18967 Xnorm[83632]:   92    Cnorm[18967]:   1.0      dists[18967]:   93.0
 Zero Division Error Encountered  - row:83721   j: 18967 Xnorm[83721]:   92    Cnorm[18967]:   1.0      dists[18967]:   93.0
 Zero Division Error Encountered  - row:83731   j: 24670 Xnorm[83731]:   91    Cnorm[24670]:   1.0      dists[24670]:   92.0
 Zero Division Error Encountered  - row:83886   j: 18967 Xnorm[83886]:   92    Cnorm[18967]:   1.0      dists[18967]:   93.0
 Zero Division Error Encountered  - row:83891   j: 24670 Xnorm[83891]:   91    Cnorm[24670]:   1.0      dists[24670]:   92.0


In [12]:
print(type(cl_hier), cl_hier.shape)
# cl_hier.min()

<class 'numpy.ndarray'> (423737,)


In [13]:
np.save(args.out, cl_hier)
print(f"Saved clusters into '{args.out}'.")

Saved clusters into 'output/chembl_29/chembl_29_clustering.npy'.


### Investigate X file

In [50]:
print(f"Loading '{args.x}'.")
# X = sc.load_sparse(args.x).tocsr()
X_orig = load_sparse(args.x).tocsr()
X = X_orig[:100, :15000]
print(" X.indptr : " , len(X.indptr), X.indptr)
print(" X.indices: " , len(X.indices) , X.indices)
print(X)
X

In [ ]:
print(f"X shape: {X.shape}")
cl0 = np.arange(X.shape[0])
Xc  = X

In [ ]:
for dist in dists:
    print(f"Running clustering for distance: {dist}")
    cl, cent = cluster(Xc, dist)
    Xc       = Xc[cent]
    cl0      = cl[cl0]




### `clustering()` Line by line

In [ ]:
from libcpp.vector cimport vector
import numpy as np
import scipy.sparse

In [ ]:
X_shape0 = X.shape[0]
X_shape1 = X.shape[1]
x_indptr  = X.indptr
x_indices = X.indices

print(X_shape0, X_shape1, len(x_indptr), len(x_indices))

In [ ]:
## Number of non zero columns 
Xnorm = np.array((X != 0).sum(axis = 1)).flatten()

print(Xnorm.shape)
print(Xnorm.sum())

In [ ]:
dists = np.zeros(X.shape[0], dtype=np.float64)
print(dists.shape)

In [ ]:
clusters = np.zeros(X.shape[0], dtype = int) - 1
perm_ids = np.random.permutation(X.shape[0])       

print(clusters)
print(perm_ids)

In [ ]:

for i in range(X_shape0):
    if i % 10000 == 0:
        print(f"Row {i}.")
    row = perm_ids[i]

    ## computing distances to all centers
    dists[0 : Cnorm.size()] = 0.0
    for j in range(x_indptr[row], x_indptr[row+1]):
        col = x_indices[j]
        for k in range(centers[col].size()):
            dists[ centers[col][k] ] += 1.0

    closest = -1
    min_dist = radius
    for j in range(Cnorm.size()):
        dists[j] = 1.0 - dists[j] / (Xnorm[row] + Cnorm[j] - dists[j])
        if dists[j] < min_dist:
            min_dist = dists[j]
            closest  = j

    if closest >= 0:
        clusters[row] = closest
        continue

    ## create a new cluster
    k = Cnorm.size()
    for j in range(x_indptr[row], x_indptr[row+1]):
        centers[ x_indices[j] ].push_back(k)
    clusters[row] = k
    Cnorm.push_back(Xnorm[row])
    center_ids.push_back(row)

print("Reassigning compounds to the closest clusters.")
num_changed = 0
for row in range(X_shape0):
    if row % 10000 == 0:
        print(f"Row {row}.")
    ## compute distances to all clusters, assign to the closest
    for j in range(x_indptr[row], x_indptr[row+1]):
        col = x_indices[j]
        for k in range(centers[col].size()):
            dists[ centers[col][k] ] += 1.0

    closest = -1
    min_dist = radius + 1e-5
    for j in range(Cnorm.size()):
        tmp_dist = 1.0 - dists[j] / (Xnorm[row] + Cnorm[j] - dists[j])
        if tmp_dist < min_dist:
            min_dist = tmp_dist
            closest  = j
            if min_dist == 0:
                ## best possible
                break
    if (closest >= 0) and (clusters[row] != closest):
        clusters[row]  = closest
        num_changed   += 1

print(f"Reassignement changed {num_changed} assignments.")
print(f"Total {len(center_ids)} clusters.")

return np.asarray(clusters), np.asarray(center_ids)